In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

diagnostics_file = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
diagnostics_df = pd.read_excel(diagnostics_file)

# Rename "SA" to "SI" in the "Rhythm" column
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].replace("SA", "SI")

# Encode "Gender" column: 0 for "MALE" and 1 for "FEMALE"
diagnostics_df["Gender"] = diagnostics_df["Gender"].map({"MALE": 0, "FEMALE": 1})

# Separate features and labels
features = diagnostics_df.drop(columns=["FileName", "Rhythm", "Beat"]).values
labels = diagnostics_df["Rhythm"].values

# Convert features to float32
features = features.astype("float32")

# Encode labels as one-hot
unique_labels = np.unique(labels)
label_map = {label: index for index, label in enumerate(unique_labels)}
labels_encoded = to_categorical([label_map[label] for label in labels])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

print(X_train.shape, labels_encoded.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

2024-11-08 22:12:49.846202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 22:12:49.857124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 22:12:49.860439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 22:12:49.869391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 22:12:50.410722: W tensorflow/compiler/tf2

(8516, 13) (10646, 11)
(8516, 13) (8516, 11)
(2130, 13) (2130, 11)


In [2]:
# Defining the MLP model architecture
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(labels_encoded.shape[1], activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Set up early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
print(y_pred_classes[0], y_test_classes[0])

# Map back to original labels
label_names = [label for label, index in sorted(label_map.items(), key=lambda item: item[1])]
print(label_names)
print("\nClassification Report:\n")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_names))

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1731082372.230313  303771 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731082372.264575  303771 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731082372.

Epoch 1/20


I0000 00:00:1731082372.926816  303925 service.cc:146] XLA service 0x722e7c00a480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731082372.926843  303925 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-08 22:12:52.941487: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-08 22:12:53.013538: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


148/267 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3476 - loss: 47.9528

I0000 00:00:1731082373.637088  303925 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


267/267 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4376 - loss: 32.1992 - val_accuracy: 0.6831 - val_loss: 1.4723
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 0.6995 - loss: 1.3986 - val_accuracy: 0.7047 - val_loss: 1.3552
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.6930 - loss: 1.2337 - val_accuracy: 0.7254 - val_loss: 1.0370
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 0.7217 - loss: 1.0217 - val_accuracy: 0.7131 - val_loss: 0.9978
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 0.7315 - loss: 0.9832 - val_accuracy: 0.7380 - val_loss: 1.0231
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - accuracy: 0.7343 - loss: 0.9610 - val_accuracy: 0.7427 - val_loss: 1.0730
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.7244 - loss: 0.9960 - val_accuracy: 0.7854 - val_loss: 0.8843
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.7370 - loss: 0.9584 - val_accuracy

ValueError: Number of classes, 10, does not match size of target_names, 11. Try specifying the labels parameter